In [12]:
#instalar libreria optuna
!pip install optuna

El objetivo del programa es encontrar los mejores parámetros para un modelo de Random Forest. Para lograrlo usamos Optuna, que se encarga de explorar y seleccionar las combinaciones que ofrecen el mejor rendimiento.

Optuna usa una optimización estocástica, donde los hiperparámetros se van probando de forma aleatoria o adaptativa según lo que va aprendiendo. Además, emplea optimización bayesiana para modelar cómo se comporta la función objetivo y así dirigir la búsqueda hacia las zonas donde es más probable obtener mejores resultados. A la vez, va descartando los trials que rinden mal, para no seguir perdiendo tiempo en configuraciones poco prometedoras.

In [13]:
# codigo para descubrir los mejores hiperparametros
import pandas as pd
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna.visualization as vis
import plotly.graph_objects as go

# cargar datos
df = pd.read_csv('dataset.csv')
X = df.drop(['id','target_variable'], axis=1)
y = df['target_variable']

# rangos de busqueda de los hiperparámetros
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 10)  # mínimo 5 para evitar overfitting
    max_features = trial.suggest_categorical('max_features', ['sqrt','log2', None])

    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1  # usamos todos los núcleos
    )

    # 3 folds del cross validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(rf, X, y, cv=cv, scoring='accuracy', n_jobs=-1).mean()
    return score

# crear y ejecutar el estudio
study = optuna.create_study(direction='maximize')  # se busca maximizar la acuracy
study.optimize(objective, n_trials=30)  # 30 evaluaciones de hiperparámetros

# filtrar y mostrar solo los resultados con score > 0.8
best_trials = [trial for trial in study.trials if trial.value is not None and trial.value > 0.8]

if best_trials: # comprobar si la lista best_trials no está vacía.

    # ordenar best_trials por su value de mayor a menor.
    best_trials.sort(key=lambda x: x.value, reverse=True)

    print("Trials con accuracy > 0.8:")
    for i, trial in enumerate(best_trials[:3]):  # Mostrar top 3
        print(f"\nTrial #{i+1}:")
        print(f"Score: {trial.value:.4f}")
        print(f"Hiperparámetros: {trial.params}")

    print(f"\nTotal de trials con accuracy > 0.8: {len(best_trials)}")
    print(f"\nMejores hiperparámetros: {study.best_params}")
    print(f"Mejor score CV: {study.best_value:.4f}")

else:
    print("No se encontraron trials con accuracy mayor a 0.8")
    print(f"Mejor score encontrado: {study.best_value:.4f}")


# GRAFICAS
# 1. Mejor score acumulado
scores = [trial.value for trial in study.trials]
cummax_scores = [max(scores[:i+1]) for i in range(len(scores))]
fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=scores, mode='markers', name='Score por trial'))
fig1.add_trace(go.Scatter(y=cummax_scores, mode='lines+markers', name='Mejor score acumulado', line=dict(color='red', width=2)))
fig1.update_layout(title="1. Convergencia con Mejor Score Acumulado",
                   xaxis_title="Número de Trial",
                   yaxis_title="Accuracy",
                   yaxis=dict(range=[0.7, 1.0]))
fig1.show()

# 2. Parallel Coordinate
fig2 = vis.plot_parallel_coordinate(study)
fig2.update_layout(title="2. Parallel Coordinate: Influencia de Hiperparámetros")
fig2.show()

# 3. Importance Plot
fig3 = vis.plot_param_importances(study)
fig3.update_layout(title="3. Importancia de los Hiperparámetros")
fig3.show()



[I 2025-11-18 18:53:24,772] A new study created in memory with name: no-name-f1f0ba61-4488-4140-8a61-e845da82753c
[I 2025-11-18 18:54:09,312] Trial 0 finished with value: 0.7534749220262494 and parameters: {'n_estimators': 361, 'max_depth': 42, 'min_samples_split': 14, 'min_samples_leaf': 8, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7534749220262494.
[I 2025-11-18 18:54:32,631] Trial 1 finished with value: 0.7409397585518512 and parameters: {'n_estimators': 404, 'max_depth': 23, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7534749220262494.
[I 2025-11-18 18:54:43,159] Trial 2 finished with value: 0.7463159815926096 and parameters: {'n_estimators': 170, 'max_depth': 16, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7534749220262494.
[I 2025-11-18 18:56:06,587] Trial 3 finished with value: 0.7909689833012147 and parameters: {'n_estimators': 412, 'max_depth': 24,

Trials con accuracy > 0.8:

Trial #1:
Score: 0.8156
Hiperparámetros: {'n_estimators': 386, 'max_depth': 45, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_features': None}

Trial #2:
Score: 0.8152
Hiperparámetros: {'n_estimators': 444, 'max_depth': 44, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': None}

Trial #3:
Score: 0.8151
Hiperparámetros: {'n_estimators': 490, 'max_depth': 50, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': None}

Total de trials con accuracy > 0.8: 17

Mejores hiperparámetros: {'n_estimators': 386, 'max_depth': 45, 'min_samples_split': 7, 'min_samples_leaf': 5, 'max_features': None}
Mejor score CV: 0.8156


### Conclusiones

- **max_depth** es el hiperparámetro más importante.
- **max_features** es el segundo más importante.
- **max_features = None** genera casi todos los mejores resultados (>0.81).
- El **max_depth óptimo** está entre **25 y 35**.
- **min_samples_leaf = 5** aparece en todos los mejores modelos.
- **n_estimators** entre **180 y 230** funciona igual que usar 400–500, por lo que tiene poca importancia.
- La **accuracy no mejora** usando *sqrt* o *log2* como max_features.

---

### Acciones a tomar

- Ajustar los rangos de los hiperparámetros más importantes.
- Reducir **max_depth** al rango **20–40**.
- Fijar **max_features = None**.
- Fijar **min_samples_leaf = 5**.
- Reducir el rango de **n_estimators**, ya que entre **180–240** funciona igual que valores más altos.
- Afinar **min_samples_split** alrededor de **8–9**.


In [16]:
# codigo para descubrir los mejores hiperparametros optimizado por optuna
import pandas as pd
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna.visualization as vis
import plotly.graph_objects as go

# cargar datos
df = pd.read_csv('dataset.csv')
X = df.drop(['id','target_variable'], axis=1)
y = df['target_variable']

# rangos de busqueda de los hiperparámetros
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 150, 300)         # modificado
    max_depth = trial.suggest_int('max_depth', 20, 40)                 # modificado
    min_samples_split = trial.suggest_int('min_samples_split', 5, 12)  # modificado
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 10)    # mínimo 5 para evitar overfitting
    max_features = trial.suggest_categorical('max_features', ['sqrt','log2', None])

    rf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=5,         #fijado
        max_features=None,          #fijado
        class_weight='balanced',
        random_state=42,
        n_jobs=-1  # usamos todos los núcleos
    )

    # 3 folds del cross validation
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    score = cross_val_score(rf, X, y, cv=cv, scoring='accuracy', n_jobs=-1).mean()
    return score

# crear y ejecutar el estudio
study = optuna.create_study(direction='maximize')  # se busca maximizar la acuracy
study.optimize(objective, n_trials=30)  # 30 evaluaciones de hiperparámetros

# filtrar y mostrar solo los resultados con score > 0.8
best_trials = [trial for trial in study.trials if trial.value is not None and trial.value > 0.8]

if best_trials: # comprobar si la lista best_trials no está vacía.

    # ordenar best_trials por su value de mayor a menor.
    best_trials.sort(key=lambda x: x.value, reverse=True)

    print("Trials con accuracy > 0.8:")
    for i, trial in enumerate(best_trials[:3]):  # Mostrar top 3
        print(f"\nTrial #{i+1}:")
        print(f"Score: {trial.value:.4f}")
        print(f"Hiperparámetros: {trial.params}")

    print(f"\nTotal de trials con accuracy > 0.8: {len(best_trials)}")
    print(f"\nMejores hiperparámetros: {study.best_params}")
    print(f"Mejor score CV: {study.best_value:.4f}")

else:
    print("No se encontraron trials con accuracy mayor a 0.8")
    print(f"Mejor score encontrado: {study.best_value:.4f}")


# GRAFICAS
# 1. Mejor score acumulado
scores = [trial.value for trial in study.trials]
cummax_scores = [max(scores[:i+1]) for i in range(len(scores))]
fig1 = go.Figure()
fig1.add_trace(go.Scatter(y=scores, mode='markers', name='Score por trial'))
fig1.add_trace(go.Scatter(y=cummax_scores, mode='lines+markers', name='Mejor score acumulado', line=dict(color='red', width=2)))
fig1.update_layout(title="1. Convergencia con Mejor Score Acumulado",
                   xaxis_title="Número de Trial",
                   yaxis_title="Accuracy",
                   yaxis=dict(range=[0.7, 1.0]))
fig1.show()

# 2. Parallel Coordinate
fig2 = vis.plot_parallel_coordinate(study)
fig2.update_layout(title="2. Parallel Coordinate: Influencia de Hiperparámetros")
fig2.show()

# 3. Importance Plot
fig3 = vis.plot_param_importances(study)
fig3.update_layout(title="3. Importancia de los Hiperparámetros")
fig3.show()


[I 2025-11-18 19:27:12,864] A new study created in memory with name: no-name-2019ad69-9b18-480d-bebf-c9c9179e3a56
[I 2025-11-18 19:27:54,642] Trial 0 finished with value: 0.8139779745945827 and parameters: {'n_estimators': 182, 'max_depth': 38, 'min_samples_split': 11, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 0 with value: 0.8139779745945827.
[I 2025-11-18 19:28:41,067] Trial 1 finished with value: 0.8143958205156293 and parameters: {'n_estimators': 209, 'max_depth': 29, 'min_samples_split': 7, 'min_samples_leaf': 10, 'max_features': 'log2'}. Best is trial 1 with value: 0.8143958205156293.
[I 2025-11-18 19:29:33,348] Trial 2 finished with value: 0.8104681275181868 and parameters: {'n_estimators': 257, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 10, 'max_features': None}. Best is trial 1 with value: 0.8143958205156293.
[I 2025-11-18 19:30:24,024] Trial 3 finished with value: 0.8117773461024355 and parameters: {'n_estimators': 208, 'max_depth': 25, 'mi

Trials con accuracy > 0.8:

Trial #1:
Score: 0.8150
Hiperparámetros: {'n_estimators': 194, 'max_depth': 33, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': 'sqrt'}

Trial #2:
Score: 0.8149
Hiperparámetros: {'n_estimators': 194, 'max_depth': 37, 'min_samples_split': 6, 'min_samples_leaf': 5, 'max_features': 'sqrt'}

Trial #3:
Score: 0.8147
Hiperparámetros: {'n_estimators': 189, 'max_depth': 33, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': 'sqrt'}

Total de trials con accuracy > 0.8: 30

Mejores hiperparámetros: {'n_estimators': 194, 'max_depth': 33, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': 'sqrt'}
Mejor score CV: 0.8150


Aunque en cada iteración Optuna va proponiendo combinaciones de hiperparámetros que tienden a mejorar el rendimiento del modelo, en este proyecto todavía voy ajustando manualmente los rangos y comprobando los resultados visualizando las gráficas generadas.

Soy consciente de que todo este proceso se puede automatizar por completo (ajuste dinámico de rangos, relanzar estudios según resultados previos, selección automática de hiperparámetros, etc.). Lo implementaré en futuros proyectos, pero en esta versión he preferido mantener control manual para entender mejor cómo se comporta el modelo.